In [1]:
import hydraseq as hds
from hydraseq import run_convolutions #, flatten_tree

In [2]:
sentence = [["spring"], ["leaves"], ["spring"]]
hd0 = hds.Hydraseq("0_")
hd0.insert(sentence)
hd0.columns

defaultdict(list,
            {'leaves': [<node: leaves,(*) spring leaves>],
             'spring': [<node: spring,(*) spring>,
              <node: spring,(*) spring leaves spring>]})

In [3]:
hd1 = hds.Hydraseq("1_")
encodes = [
    "spring 1_Adj",
    "spring 1_Verb",
    "spring 1_Noun",
    "leaves 1_Noun",
    "leaves 1_Verb"
]
for encode in encodes:
    hd1.insert(encode)

In [4]:
hd1.columns

defaultdict(list,
            {'1_Adj': [<node: 1_Adj,(*) spring 1_Adj>],
             '1_Noun': [<node: 1_Noun,(*) spring 1_Noun>,
              <node: 1_Noun,(*) leaves 1_Noun>],
             '1_Verb': [<node: 1_Verb,(*) spring 1_Verb>,
              <node: 1_Verb,(*) leaves 1_Verb>],
             'leaves': [<node: leaves,(*) leaves>],
             'spring': [<node: spring,(*) spring>]})

In [5]:
conv1 = run_convolutions("spring leaves spring", hd1)
conv1

[[0, 1, ['1_Adj', '1_Noun', '1_Verb']],
 [1, 2, ['1_Noun', '1_Verb']],
 [2, 3, ['1_Adj', '1_Noun', '1_Verb']]]

In [6]:
def generate_tree(lst_nods, debug=False):
    print("generate_tree")
    seq = hds.Hydraseq("_")
    seq.insert("0")
    zero = seq.columns["0"][0]
    for idx, nod in enumerate(lst_nods):
        node = hds.Node(nod[2])
        for nd in seq.columns[str(nod[0])]:
            print("nd from ", str(nod[0]))
            nd.nexts.append(node)
            node.lasts.append(nd)
            if debug: print("A inserting a node at {}".format(str(nod[1])))
            seq.columns[str(nod[1])].append(node)
            break;
        else:
            zero.nexts.append(node)
            node.lasts.append(zero)
            if debug: print("B inserting a node at {}".format(str(nod[1])))
            seq.columns[str(nod[1])].append(node)
        
    return seq
    

In [7]:
s = generate_tree(conv1)
s.columns

generate_tree
nd from  0
nd from  1
nd from  2


defaultdict(list,
            {'0': [<node: 0,(*) 0>],
             '1': [<node: ['1_Adj', '1_Noun', '1_Verb'],(*) 0 ['1_Adj', '1_Noun', '1_Verb']>],
             '2': [<node: ['1_Noun', '1_Verb'],(*) 0 ['1_Adj', '1_Noun', '1_Verb'] ['1_Noun', '1_Verb']>],
             '3': [<node: ['1_Adj', '1_Noun', '1_Verb'],(*) 0 ['1_Adj', '1_Noun', '1_Verb'] ['1_Noun', '1_Verb'] ['1_Adj', '1_Noun', '1_Verb']>]})

In [8]:
def flatten_tree(seq, debug=False):
    print("flatten_tree")
    sequences = []
    for node_tup in [(key, node) for key, lst_nodes in seq.columns.items() for node in lst_nodes if not node.nexts]:
        key, node = node_tup
        print(key, node.get_sequence())
        outcome = " ".join(node.get_sequence().split()[2:])
        found_list = list(eval(outcome.replace("] [", "], [")))
        sequences.append(found_list)
    return sequences

flattened = flatten_tree(s, debug=True)
print(flattened)

flatten_tree
3 (*) 0 ['1_Adj', '1_Noun', '1_Verb'] ['1_Noun', '1_Verb'] ['1_Adj', '1_Noun', '1_Verb']
[[['1_Adj', '1_Noun', '1_Verb'], ['1_Noun', '1_Verb'], ['1_Adj', '1_Noun', '1_Verb']]]


## Problem, generate tree inserts 6 nodes, flatten tree identifies 5..

In [20]:
hd2 = hds.Hydraseq('_')
for pattern in [
    "1_Noun _NP_",
    "1_Adj 1_Noun _NP_",
    "1_Verb _VP_",
    "1_Adj 1_Noun _NP_",
    "1_Adj 1_Adj 1_Adj 1_Noun _NP_",

]:
    hd2.insert(pattern)
    
for flat in flattened:
    convo = run_convolutions(flat, hd2)
    print(convo)
    f = generate_tree(convo)
    for column, vals in f.columns.items():
        print(column, vals)
    print(flatten_tree(f))

[[0, 1, ['_NP_', '_VP_']], [0, 2, ['_NP_']], [1, 2, ['_NP_', '_VP_']], [2, 3, ['_NP_', '_VP_']]]
generate_tree
nd from  0
nd from  0
nd from  1
nd from  2
0 [<node: 0,(*) 0>]
2 [<node: ['_NP_'],(*) 0 ['_NP_']>, <node: ['_NP_', '_VP_'],(*) 0 ['_NP_', '_VP_'] ['_NP_', '_VP_']>]
3 [<node: ['_NP_', '_VP_'],(*) 0 ['_NP_'] ['_NP_', '_VP_']>]
1 [<node: ['_NP_', '_VP_'],(*) 0 ['_NP_', '_VP_']>]
flatten_tree
2 (*) 0 ['_NP_', '_VP_'] ['_NP_', '_VP_']
3 (*) 0 ['_NP_'] ['_NP_', '_VP_']
[[['_NP_', '_VP_'], ['_NP_', '_VP_']], [['_NP_'], ['_NP_', '_VP_']]]


In [ ]:

from hydraseq import Hydraseq, run_convolutions


hdq1 = Hydraseq('0_')
for pattern in [
    "the 0_A",
    "quick 0_ADJ",
    "brown 0_ADJ",
    "fox 0_N",
    "fox 0_V",
    "jumped 0_V",
    "over 0_PR",
    "lazy 0_ADJ",
    "dog 0_N"
]:
    hdq1.insert(pattern)

hdq2 = Hydraseq('_')
for pattern in [
    "0_N _NP_",
    "0_ADJ 0_N _NP_",
    "0_V _VP_",
    "0_ADV 0_V _VP_",
    "0_A 0_N _NP_",
    "0_A 0_ADJ 0_ADJ 0_N _NP_",

]:
    hdq2.insert(pattern)

hdq3 = Hydraseq('3_')
for pattern in [
    "_NP_ _VP_ 3_BINGO"
]:
    hdq3.insert(pattern)

sentence = "the quick brown fox jumped over the lazy dog"

hdq0 = Hydraseq('0_')
for word in sentence.split():
    hdq0.insert("{} 0_{}".format(word,word))
#print(run_convolutions(sentence.split(), hdq0))
result = run_convolutions(sentence.split(), hdq1)
expected = [[0, 1, ['0_A']], [1, 2, ['0_ADJ']], [2, 3, ['0_ADJ']], [3, 4, ['0_N', '0_V']], [4, 5, ['0_V']], [5, 6, ['0_PR']], [6, 7, ['0_A']], [7, 8, ['0_ADJ']], [8, 9, ['0_N']]]
assert result == expected
s = generate_tree(result)
encodeds = flatten_tree(s)
assert encodeds == [[['0_A'], ['0_ADJ'], ['0_ADJ'], ['0_N', '0_V'], ['0_V'], ['0_PR'], ['0_A'], ['0_ADJ'], ['0_N']]]

results = run_convolutions(encodeds[0], hdq2)
expected = [[0, 4, ['_NP_']], [2, 4, ['_NP_']], [3, 4, ['_NP_', '_VP_']], [4, 5, ['_VP_']], [7, 9, ['_NP_']], [8, 9, ['_NP_']]]
assert results == expected
s = generate_tree(results, debug=True)
encodeds = flatten_tree(s)
print(encodeds)
assert encodeds == [['_NP_'], ['_NP_', '_VP_'], [['_NP_'], ['_VP_']], ['_NP_'], ['_NP_']]
print(results)
for key, nodes in s.columns.items():
    print(key, nodes)


expected = [[0,2, ['3_BINGO']]]
results = run_convolutions(encodeds, hdq3)
print(results)
assert results == expected

In [ ]:
hdq2.look_ahead([['0_A'], ['0_ADJ'], ['0_ADJ'], ['0_N', '0_V']]).get_next_values()

In [ ]:
hydra = Hydraseq(1, hdq2)

In [ ]:
hydra.hit(['0_N', '0_V'], is_learning=False).get_next_values()